# Load data
cats = pickle.load(open('./all_cats.pickle', 'rb'))
for cnt, cat in enumerate(cats):
    ind = np.isfinite(cat['eps'])
    lambda_e.extend( cat['lambda_r'][ind])
    eps.extend(cat['eps'][ind])

    lambda_c.extend( cat['lambda_r'][ind])
    stellarmass.extend( cat['mstar'][ind])
    reff.extend( cat['reff'][ind])
    ids.extend( cat['id'][ind] + cnt*10000) # annotation!
    d2t.extend( cat['d2t'][ind])

    rank.extend( 100*(np.argsort(cat['mstar'][ind])/sum(ind) ))

In [24]:
import pickle
import matplotlib.pyplot as plt 
import numpy as np

do_lambda_vs_e = True
do_scatter = True
sami_scatter = False
load_data = False

eps = []
lambda_c = []
lambda_e = []
stellarmass =[]
reff = []
ids = []
d2t = []

rank = []

annotate=False # output id of flat slow rotators
color = False # BCGs in different colors.
nout_fi = 37

if load_data:
    mpgs = pickle.load(open("main_prgs_final_augmented_5_10_0.5_0.5_0.5_37_0.01_filtered_.pickle", "rb"))
    eps = []
    lambda_e = []

    inout = 0 # nout=187
    mstar_cut_hard = 5e9

    for gal in mpgs:
        mgal = gal.data["mstar"][0]
        if mgal > mstar_cut_hard:
            eps.append(gal.data['eps'][inout])
            lambda_e.append(gal.data['lambda_r'][inout])    
else:
    fname_vs_e = "./figs/lambda_vs_e_HM_z1"
    nout_fi = [782, 358, 125][2]
    cat_dir = ["easy_final/"]              
    clusters = [['07206', '04466', '01605', \
                '35663', '24954', '49096', \
                '05427', '05420', '29172', \
                '29176', '10002', '36415', \
                '06098', '39990', '36413', \
                '17891'],[0,1,2,3,4,6,7,8,9,10,11,15,16]][1]
                #[0,1,2,3,4,6,7,8,9,10,11,12,13,15,16,17,18,19,20]][1]
    clusters = [str(i) for i in clusters]
    #clusters = ["05420"]
                # 35663, 49096, 249554, and 14172 to be added.

    from analysis import misc
    for cnt, cluster in enumerate(clusters):
        print(cluster)
        #fname = cluster + "/" + cat_dir + "catalog" + str(nout_fi) + ".pickle"
        fname = "Horizon-AGN/lambda_results/" + cluster + "/" + "catalog" + str(nout_fi) + cluster + ".pickle"
        cat = misc.load_cat(fname)
        cat = cat[~np.isnan(cat['eps'])]

        ind = np.isfinite(cat['eps'])
        lambda_e.extend(cat['lambda_r'][ind])
        eps.extend(cat['eps'][ind])

        lambda_c.extend(cat['lambda_r'][ind])
        stellarmass.extend(cat['mstar'][ind])
        reff.extend(cat['reff'][ind])
        ids.extend(cat['id'][ind] + cnt*10000) # annotation!
        d2t.extend(cat['d2t'][ind])

        rank.extend(100*(np.argsort(cat['mstar'][ind])/sum(ind)))
        #print(cat['eps'])


0
1
2
3
4
6
7
8
9
10
11
15
16


In [28]:
len(eps)

6852

In [29]:
i = np.isfinite(lambda_e)

In [30]:
sum(i)

6848

In [32]:
# load SAMI data
from utils import match
from analysis.all_plot_modules import *

sd = np.genfromtxt('./samidata/jvds_sami_lambdar_220216_0arc_fwhm.txt',
                    dtype=(int, float, float, float, float, float, float),
                    usecols=(0,2,3,6,7,10,11))
sd.dtype.names = ('id', 'r05', 'e05', 'r1', 'e1', 'r2', 'e2') # e = errors, ellip = ellipticity

dd = np.genfromtxt('./samidata/input_catalog_lambdar.txt',
                          dtype=(int, float, float, float),
                          usecols=(0,3,4,6))
dd.dtype.names = ('id', 're', 'ellp', 'mstar')

common_ids = np.intersect1d(sd['id'], dd['id'])

ind = match.match_list_ind(common_ids, sd['id'])
sd = sd[ind]
ind = match.match_list_ind(common_ids, dd['id'])
dd = dd[ind]

from numpy.lib.recfunctions import append_fields

if all(sd['id'] == dd['id']):
    tt = dd[['re', 'ellp', 'mstar']]
    sami_data = append_fields(sd, tt.dtype.names, [tt[n] for n in tt.dtype.names])
    # only valid values
    all_finite = np.all([np.isfinite(sami_data[field]) for field in ['id', 'r05', 'r1', 're', 'ellp', 'mstar']], axis=0)
    sami_data = sami_data[all_finite]
    
# exclude ellp == 0

sami_data = sami_data[sami_data['ellp'] > 0].data

# Mass cut
sami_data = sami_data[sami_data['mstar'] > 9.7]


atlas = np.genfromtxt('./ATLAS3D/Emsellem2011_Atlas3D_Paper3_TableB1.txt',
                      skip_header=12,
                      usecols=(2,7))

x = np.array(eps)[i] # isfinit
y = np.array(lambda_e)[i]
print("number of galaxies in total", len(eps))
print("Number of galaxies below the demarkation line:", sum(y < 0.31 * np.sqrt(x)))

#new_map = truncate_colormap("winter_r", minval=0.3)

# Color
#hex_colors = ['#4c72b0', '#55a868', '#c44e52', '#8172b2', '#ccb974', '#64b5cd']
twocolors=['#4c72b0', '#c44e52']
#twocolors=["blue", "red"]
den_cmap=["cool", "YlGnBu", "PuBu"][1]
#blue = '#4c72b0'
#red = '#c44e52'
do_plot(x,y, atlas,
        do_scatter=False,
        contour_label=False,
        surf = False,
        img_scale = 2.0,
        twocolors=twocolors,
        den_cmap = "YlGnBu_r",
        d_alpha=1.0,
        levels=None,#np.linspace(0.02, 1.0, 19),
        fname_vs_e = "./figs/lambda_vs_e_HM")

number of galaxies in total 6852
Number of galaxies below the demarkation line: 64


In [72]:
# components
do_scatter=True
contour_label=False
surf = False


import itertools
for levels in [np.linspace(0.01, 1.0, 19), None]:
    for combination in itertools.product(*[(True, False)]*3):
        do_scatter, contour_label, surf = combination
        do_plot(x,y, 
                do_scatter=do_scatter,
                contour_label=contour_label,
                surf = surf,
                img_scale = 2.0,
                twocolors=twocolors,
                den_cmap = "PuBu",
                levels=levels,
                fname_vs_e = "./figs/lambda_vs_e_z0")